In [3]:
%%writefile SDTW.py
import numpy
from pandas import read_csv
from pandas import *
import time
#open and clean the dataset

#input dataset
filename='daily_O3_2019.csv'
names=['Latitude','Longitude','Date Local','Arithmetic Mean']
dataset=read_csv(filename,names=names)

dataset101=dataset.drop_duplicates(subset=['Latitude'])
dataset1=dataset.pivot_table(index = 'Date Local',columns = 'Latitude',values = 'Arithmetic Mean')#transform long data format into wide format
dataset11=dataset1.dropna(axis=1, how='any')#delete sites which includes NaN data
dataset111=dataset11.values.T
print(dataset11)

#get information of latitudes and longitudes as lists
latitudes=dataset11.columns.values.tolist()
longitudes=[]
for i in range(len(latitudes)):
    long1=dataset101.Longitude[dataset101['Latitude'] == latitudes[i]].tolist()
    long2=float(long1[0])
    longitudes.append(long2)


#X_train prepared
from tslearn.utils import to_time_series_dataset
formatted_dataset = to_time_series_dataset(dataset111)#formatted
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
import matplotlib.pyplot as plt
X_train = formatted_dataset
X_train = TimeSeriesScalerMeanVariance().fit_transform(X_train)#normalized
sz = X_train.shape[1]


from mpl_toolkits.basemap import Basemap 
import matplotlib.pyplot as plt  
import multiprocessing

#2019,PM2.5,cluster=3,seed=0
#training—TimeSeriesKMeans
def SDKM(n_cluster,seed):
    print("Soft-DTW k-means")
    sdtw_km = TimeSeriesKMeans(n_cluster,
                               metric="softdtw",
                               metric_params={"gamma": .01},
                               verbose=True,
                               random_state=seed)
    y_pred = sdtw_km.fit_predict(X_train)
    y1=y_pred.tolist()

    #draw the time series clusters
    plt.figure()
    for yi in range(n_cluster):
        plt.subplot(n_cluster ,1, yi + 1)  #plt.subplot（numbRow，numbCol ，plotNum）
        for xx in X_train[y_pred == yi]:
            plt.plot(xx.ravel(), "k-", alpha=.2,zorder=1) #k-:black
        plt.plot(sdtw_km.cluster_centers_[yi].ravel(), "r-",zorder=5)
        plt.hlines(0,0,365,color='#FFD700',zorder=10)#draw the average guideline
        plt.xlim(0, sz)
        plt.ylim(-2, 4)
        plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
                 transform=plt.gca().transAxes)
        if yi == 0:
            plt.title('2019,O3,Soft-DTW k-means,cluster:%d,seed:%d'% (n_cluster,seed))
    plt.subplots_adjust(hspace=0.3) #adjust the vertical space between subplots
    plt.savefig('1-2019-O3-SDKM-cluster %d-seed %d.jpg'% (n_cluster,seed))
    plt.show()
    #draw the clutering results
    
    #draw the map
    fig=plt.figure(figsize=(7.6, 3.8)) 
    m= Basemap(llcrnrlon=-130, llcrnrlat=20, urcrnrlon=-65, urcrnrlat=49,projection='lcc',lat_1=24, lat_2=45, lon_0=-100)
    m.drawcountries(linewidth=1.5) 
    m.drawcoastlines()  
    m.drawstates()       
    #draw the datapoints
    #lon, lat = m(longitudes,latitudes)# longitudes, latitudes in the form of lists 
    #m.scatter(lon,lat,c=y1)
    color=['#FFFF00','#00BFFF','#4B0082','#8B0000','#FFA500', '#EE82EE']
    for j in range(n_cluster):
        lons=[]
        lats=[]
        for i in range(len(y1)):
            if y1[i]==j:
                lons.append(longitudes[i])
                lats.append(latitudes[i])
        lon, lat = m(lons,lats)
        m.scatter(lon,lat,c=color[j],label='Cluster %d' % (j + 1),s=8)#mark the points on the map
    plt.legend()
    plt.title('2019,O3,Soft-DTW k-means,cluster:%d,seed:%d'% (n_cluster,seed))
    plt.savefig('2019-O3-SDKM-cluster %d-seed %d.jpg'% (n_cluster,seed))
    plt.show()



if __name__ == '__main__':
    start=time.time()
    __spec__ = "ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>)"
    p1 = multiprocessing.Process(target=SDKM, args=(3,0))
    p2 = multiprocessing.Process(target=SDKM, args=(3,1))
    p3 = multiprocessing.Process(target=SDKM, args=(3,2))
    p1.start() 
    p2.start()
    p3.start()
    p1.join()
    p2.join()
    p3.join()
    end=time.time()
    print((end-start)/60)




Overwriting SDTW.py


In [4]:
%run SDTW.py

Latitude    19.203900  19.509778  19.555444  21.390861  27.939746  28.141667  \
Date Local                                                                     
43466        1.937500   1.114584   2.883333   0.139583   1.285416   0.670833   
43467        1.729167   1.133333   2.954167   0.977083   1.460065   0.675000   
43468        1.595833   1.192857   3.046894   0.402084   1.277084   0.635606   
43469        1.375000   1.025000   2.985417   0.752084   1.072917   0.631250   
43470        2.158333   0.985416   3.072671   0.058333   0.820833   0.612500   
...               ...        ...        ...        ...        ...        ...   
43826        0.656250   0.118750   2.175000   0.085417   0.195833   0.495833   
43827        0.606250   0.160416   2.087500   0.104166   0.714584   0.583333   
43828        1.093750   0.154166   2.164583   0.141667   0.331250   0.620186   
43829        1.086646   0.196212   2.133333   0.113637   0.572917   0.601190   
43830        1.286904   0.191666   2.222